## Lab 4: OpenAI for non-OpenAI

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown, update_display
import requests
load_dotenv(override=True)

True

### We're now going to ask a hard question to lots of models

In [2]:
message = "In 1 sentence, describe a rainbow to someone who's never been able to see. \
Then in 1 sentence, describe the imaginary number i to someone who doesn't understand math. \
Then in 1 sentence, find a connection between rainbows and imaginary numbers. \
Then end by stating how many words are in your answer."

messages = [{"role": "user", "content": message}]

In [3]:
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI
DeepSeek API Key exists and begins sk-
Groq API Key exists and begins gsk_
Grok API Key not set (and this is optional)


#### <span style="color: green;">Question: what's going on below? Why are we calling OpenAI with details about Anthropic?</span>

In [4]:
anthropic_url = "https://api.anthropic.com/v1"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
ollama_url = 'http://localhost:11434/v1/chat'

openai = OpenAI()
anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
deepseek = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
ollama = OpenAI(base_url=ollama_url, api_key='ollama')


In [5]:
models = []
answers = []

def answer(client, model):
    stream = client.chat.completions.create(model=model, messages=messages, stream=True)
    prefix = f"### Response from {model}:\n\n"
    reply = ""
    display_handle = display(Markdown(prefix), display_id=True)
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ''
        update_display(Markdown(prefix+reply), display_id=display_handle.display_id)
    words = reply.split('</think>')[1] if '</think>' in reply else reply
    reply += f"\n\n#### Calculated true word count: {len(words.split())}"
    update_display(Markdown(prefix+reply), display_id=display_handle.display_id)
    
    models.append(model)
    answers.append(reply)

In [6]:
answer(openai, "gpt-4.1-mini")

### Response from gpt-4.1-mini:

A rainbow is a graceful arc of many colors that appears in the sky when sunlight passes through raindrops, creating a natural spectrum of light; the imaginary number *i* is a special kind of number used in math to represent the square root of negative one, something that can't be found on a regular number line but helps solve puzzles involving direction and rotation; both rainbows and the imaginary number *i* reveal hidden layers of reality that aren't immediately visible to the naked eye, whether through light or abstract math. There are 64 words in my answer.

#### Calculated true word count: 97

In [7]:
answer(openai, "gpt-5-nano")

### Response from gpt-5-nano:

A rainbow is a curved bridge of light that appears after rain and is described as a spectrum of colors your mind can imagine or remember. 
The imaginary number i is a symbol that, when multiplied by itself, gives -1, helping describe rotations and other extensions beyond ordinary counting. 
Both rainbows and imaginary numbers invite us to think beyond what we can see or count, one through light and rain, the other through rules that extend numbers. 
Total words: 80.

#### Calculated true word count: 80

In [8]:
answer(openai, "gpt-5")

### Response from gpt-5:

A rainbow is like hearing a gentle chord after a storm—many notes blending at once—but felt as rain-cool and sun-warm touching your skin together in a soft arc.
Think of i as a step into a hidden sideways hallway of numbers, a make-believe direction we name so patterns can complete themselves when ordinary steps won't do.
Both rainbows and i appear when the straight path bends—light splitting after rain, numbers turning sideways in equations—revealing hidden structure that was always there.
81

#### Calculated true word count: 81

In [9]:
# claude-sonnet-4-20250514

answer(anthropic, "claude-sonnet-4-5-20250929")

APIConnectionError: Connection error.

In [10]:
answer(gemini, "gemini-2.5-flash-lite")

### Response from gemini-2.5-flash-lite:

A rainbow is like a beautiful, curved pathway of colors painted across the sky after rain, a warm hug of light that disappears as quickly as it appears. The imaginary number *i* is like a special kind of "what if" number that helps us solve certain math puzzles that ordinary numbers can't touch, a bit like a secret key. Both rainbows and *i* represent things that are beyond our immediate physical experience but are still incredibly useful and elegant.

There are 65 words in this answer.

#### Calculated true word count: 86

In [16]:
answer(gemini, "gemini-3-pro-preview")

### Response from gemini-3-pro-preview:

A rainbow is like the feeling of warm sun breaking through cool rain, arranged in a sweeping arch of distinct, harmonious musical notes. The imaginary number i is a conceptual tool that lets us step sideways off the standard number line to solve problems that are impossible to fix with regular numbers. These concepts connect because the complex physics of light waves bending to form a rainbow can only be fully explained using the invisible dimension of imaginary numbers. There are 86 words in this answer.

#### Calculated true word count: 86

In [12]:
# DeepSeek 3.1 Terminus - not on reasoning mode or it takes too long

answer(deepseek, "deepseek-chat")

APIConnectionError: Connection error.

In [13]:
answer(groq, "deepseek-r1-distill-llama-70b")

BadRequestError: Error code: 400 - {'error': {'message': 'The model `deepseek-r1-distill-llama-70b` has been decommissioned and is no longer supported. Please refer to https://console.groq.com/docs/deprecations for a recommendation on which model to use instead.', 'type': 'invalid_request_error', 'code': 'model_decommissioned'}}

#### <span style="color: orange;">Question: what's the difference between Grok and Groq and why do they have such similar names?</span>

In [14]:
answer(grok, "grok-4")

APIConnectionError: Connection error.

In [ ]:
!ollama pull llama3.2
!ollama pull gpt-oss

In [ ]:
requests.get("http://localhost:11434").content

In [ ]:
answer(ollama, "llama3.2")

In [ ]:
answer(ollama, "gpt-oss:20b")

In [ ]:
answer(groq, "openai/gpt-oss-120b")

In [ ]:
len(models)

## LLM as a Judge

In [ ]:
together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
display(Markdown(together))

In [ ]:
judge = f"""You are judging a competition between {len(models)} competitors.
Each model has been given this question:

{message}

Your job is to evaluate each response for clarity and strength of argument and accuracy of word count, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["best competitor number", "second best competitor number", "third best competitor number", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""

In [ ]:
display(Markdown(judge))

In [ ]:
judge_messages = [{"role": "user", "content": judge}]
response = groq.chat.completions.create(model="openai/gpt-oss-120b", messages=judge_messages)
results = response.choices[0].message.content
results


In [ ]:
results_dict = json.loads(results)
ranks = results_dict["results"]
for index, result in enumerate(ranks):
    competitor = models[int(result)-1]
    print(f"Rank {index+1}: {competitor}")